## Compliance Radar Machine Learning Project

In [27]:
# Neccessary imports to load the database and for data manipulation
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

### Loading Data from the Database

In [28]:
# Creating a cursor to connect to the SQLite database
conn = sqlite3.connect('C:\\Users\\Cesar Dushimimana\\Documents\\Compliance-Radar-ML\\data\\org_compliance_data.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)



Tables in the database: [('departments',), ('risk_summary_by_division',), ('high_risk_departments',), ('data_dictionary',)]


### Saving DB tables as CSV files for backup

In [29]:
# Reading each of the tables into pandas dataframes
departments_df = pd.read_sql_query("SELECT * FROM departments", conn)
risk_by_division_df = pd.read_sql_query("SELECT * FROM risk_summary_by_division", conn)
high_risk_departments_df = pd.read_sql_query("SELECT * FROM high_risk_departments", conn)
data_dictionary_df = pd.read_sql_query("SELECT * FROM data_dictionary", conn)

# Saving the dataframes as CSV files for easier access later folder data
departments_df.to_csv('C:\\Users\\Cesar Dushimimana\\Documents\\Compliance-Radar-ML\\data\\departments.csv', index=False)
risk_by_division_df.to_csv('C:\\Users\\Cesar Dushimimana\\Documents\\Compliance-Radar-ML\\data\\risk_summary_by_division.csv', index=False)
high_risk_departments_df.to_csv('C:\\Users\\Cesar Dushimimana\\Documents\\Compliance-Radar-ML\\data\\high_risk_departments.csv', index=False)
data_dictionary_df.to_csv('C:\\Users\\Cesar Dushimimana\\Documents\\Compliance-Radar-ML\\data\\data_dictionary.csv', index=False)

# Closing the database connection
conn.close()


### Shape of Each DB Table After Pandas DataFrame Loading

In [30]:
# Print number of rows and columns in each dataframe
print("Departments DataFrame shape:", departments_df.shape)
print("Risk by Division DataFrame shape:", risk_by_division_df.shape)
print("High Risk Departments DataFrame shape:", high_risk_departments_df.shape)
print("Data Dictionary DataFrame shape:", data_dictionary_df.shape)


Departments DataFrame shape: (709, 37)
Risk by Division DataFrame shape: (2, 8)
High Risk Departments DataFrame shape: (201, 37)
Data Dictionary DataFrame shape: (39, 4)


### Exploring Datasets

#### 1. Exploring departments dataset

In [31]:
# Describe numeric columns in departments_df
print(departments_df.describe())


       dept_age_years  manager_experience_level  supervisor_experience_level  \
count      411.000000                413.000000                   409.000000   
mean         6.802920                  2.624697                     2.398533   
std          1.915939                  1.164637                     1.137635   
min          2.000000                  0.000000                     0.000000   
25%          6.000000                  2.000000                     2.000000   
50%          7.000000                  3.000000                     2.000000   
75%          8.000000                  4.000000                     3.000000   
max         15.000000                  4.000000                     4.000000   

       reporting_lag_days  training_hours_quarterly  violations_past_3years  \
count          412.000000                409.000000              406.000000   
mean             1.682039                 43.180929                0.435961   
std              1.019693                 

In [32]:
# Exploring departments table
departments_df.head()

,dept_id,dept_name,dept_category,division,dept_type,dept_age_years,location_type,team_size,reporting_structure,manager_experience_level,...,resource_availability_score,external_interactions_frequency,risk_exposure_operational,risk_exposure_financial,operational_health_index,reporting_gaps_annual,audit_score_q1,audit_score_q2,compliance_score_final,overall_risk_score
0,DEPT-1620,Core Compliance,Financial_Reporting,Corporate_HQ,Operations,8.0,Remote,Large,Centralized,2.0,...,2.0,3.0,1.0,NaN,5.0,NaN,55.0,55.0,65.0,12.0
1,DEPT-1133,Digital Governance,Financial_Reporting,Corporate_HQ,Compliance,8.0,Urban,Large,Centralized,2.0,...,4.0,NaN,2.0,2.0,1.0,26.0,NaN,40.0,NaN,43.0
2,DEPT-4917,Central Hub,Operational_Compliance,None,None,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DEPT-6959,Compliance - Legacy Region,Financial_Reporting,Corporate_HQ,Support,8.0,Urban,Large,Centralized,4.0,...,3.0,4.0,1.0,1.0,5.0,2.0,70.0,75.0,85.0,11.0
4,DEPT-4312,West Internal Controls,Financial_Reporting,Corporate_HQ,Operations,6.0,Urban,Small,Centralized,1.0,...,4.0,2.0,1.0,1.0,5.0,2.0,45.0,45.0,45.0,11.0


In [33]:
departments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   dept_id                          709 non-null    object 
 1   dept_name                        709 non-null    object 
 2   dept_category                    709 non-null    object 
 3   division                         455 non-null    object 
 4   dept_type                        413 non-null    object 
 5   dept_age_years                   411 non-null    float64
 6   location_type                    410 non-null    object 
 7   team_size                        409 non-null    object 
 8   reporting_structure              413 non-null    object 
 9   manager_experience_level         413 non-null    float64
 10  supervisor_experience_level      409 non-null    float64
 11  primary_function                 416 non-null    object 
 12  secondary_function    

#### Check Unique and Duplicate Entries by 'dept_id'

In [35]:
# Check for duplicate dept_id rows
dup_deptid = departments_df[departments_df.duplicated(subset=['dept_id'], keep=False)].sort_values('dept_id')
print("Number of duplicate dept_id rows:", dup_deptid['dept_id'].nunique())
dup_deptid.head(10)


Number of duplicate dept_id rows: 26


,dept_id,dept_name,dept_category,division,dept_type,dept_age_years,location_type,team_size,reporting_structure,manager_experience_level,...,resource_availability_score,external_interactions_frequency,risk_exposure_operational,risk_exposure_financial,operational_health_index,reporting_gaps_annual,audit_score_q1,audit_score_q2,compliance_score_final,overall_risk_score
34,DEPT-1237,Alpha Solutions,Financial_Reporting,Corporate_HQ,Support,8.0,Urban,Large,Centralized,3.0,...,NaN,4.0,NaN,1.0,1.0,4.0,55.0,60.0,70.0,12.0
350,DEPT-1237,Central Ethics and Compliance,Financial_Reporting,Corporate_HQ,Operations,7.0,Remote,Large,Centralized,1.0,...,3.0,3.0,1.0,1.0,1.0,2.0,NaN,70.0,75.0,11.0
201,DEPT-2377,Prime Services,Financial_Reporting,Corporate_HQ,None,NaN,Urban,Small,Autonomous,4.0,...,1.0,3.0,3.0,5.0,5.0,6.0,45.0,45.0,50.0,43.0
156,DEPT-2377,South Financial Controls,Operational_Compliance,None,None,NaN,None,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,DEPT-2612,South Department,Financial_Reporting,Corporate_HQ,Operations,7.0,Remote,Large,Centralized,2.0,...,4.0,5.0,5.0,5.0,4.0,2.0,55.0,50.0,50.0,51.0
566,DEPT-2612,Prime Department,Risk_Management,Corporate_HQ,None,5.0,None,Medium,Autonomous,4.0,...,5.0,1.0,5.0,3.0,4.0,14.0,80.7,68.2,53.6,NaN
78,DEPT-2743,Compliance - Legacy Region,Financial_Reporting,Corporate_HQ,Operations,7.0,Urban,Large,Centralized,4.0,...,2.0,3.0,1.0,2.0,5.0,4.0,65.0,65.0,70.0,17.0
269,DEPT-2743,Regional Group,Financial_Reporting,Corporate_HQ,Operations,8.0,Remote,Large,Centralized,3.0,...,3.0,2.0,1.0,1.0,3.0,2.0,50.0,55.0,60.0,11.0
322,DEPT-2762,East Office,Risk_Management,Regional_Operations,None,14.0,Suburban,Medium,Hybrid,3.0,...,NaN,3.0,3.0,2.0,4.0,1.0,65.8,67.5,67.9,25.5
341,DEPT-2762,Emerging Regulatory Affairs,Financial_Reporting,Corporate_HQ,Technology,7.0,None,Large,None,2.0,...,4.0,4.0,2.0,3.0,4.0,8.0,40.0,40.0,45.0,29.0


### Resolving 'dept_id' duplication issue

In [65]:
# Identify all duplicate dept_id rows
duplicate_dept_ids = departments_df[departments_df.duplicated('dept_id', keep=False)]

# Identify rows dept_id appears exactly once
departments_unique = departments_df[~departments_df['dept_id'].isin(duplicate_dept_ids['dept_id'])]

# Save duplicate dept_id rows for futher investigation
duplicate_dept_ids.to_csv("C:\\Users\\Cesar Dushimimana\\Documents\\Compliance-Radar-ML\\data\\cleaned\\departments_duplicate_ids.csv", index=False)


### Labelling 'dept_id' Inconsistencies 'departments' dataset

#### Cross-Check With 'high_risk_departments' table

In [67]:
# dept_id in high_risk but NOT in clean_departments
missing_from_departments = high_risk_departments_df[~high_risk_departments_df['dept_id'].isin(departments_df['dept_id'])]


# 3. Intersection count
matched_count = high_risk_departments_df['dept_id'].isin(departments_df['dept_id']).sum()
print("✔ Matched dept_id:", matched_count)
print("✔ Missing dept_id from departments:", missing_from_departments.shape[0])




✔ Matched dept_id: 201
✔ Missing dept_id from departments: 0


In [62]:
missing_ids = missing_from_departments['dept_id'].unique()

# departments_df[departments_df['dept_id'].isin(missing_ids)].sort_values('dept_id')
missing_from_departments.shape

(17, 37)

In [44]:
# Creates lists of numeric and categorical columns
numeric_cols = departments_unique.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = departments_unique.select_dtypes(include=['object']).columns.tolist()
print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)


Numeric columns: ['dept_age_years', 'manager_experience_level', 'supervisor_experience_level', 'reporting_lag_days', 'training_hours_quarterly', 'violations_past_3years', 'remediation_plan_active', 'executive_support', 'external_consulting', 'engagement_programs', 'onboarding_program', 'improvement_commitment', 'digital_systems', 'external_partnerships', 'interdept_collaboration_score', 'resource_availability_score', 'external_interactions_frequency', 'risk_exposure_operational', 'risk_exposure_financial', 'operational_health_index', 'reporting_gaps_annual', 'audit_score_q1', 'audit_score_q2', 'compliance_score_final', 'overall_risk_score']
Categorical columns: ['dept_id', 'dept_name', 'dept_category', 'division', 'dept_type', 'location_type', 'team_size', 'reporting_structure', 'primary_function', 'secondary_function', 'creation_reason', 'oversight_body']


In [5]:
high_risk_departments_df.head()

,dept_id,dept_name,dept_category,division,dept_type,dept_age_years,location_type,team_size,reporting_structure,manager_experience_level,...,resource_availability_score,external_interactions_frequency,risk_exposure_operational,risk_exposure_financial,operational_health_index,reporting_gaps_annual,audit_score_q1,audit_score_q2,compliance_score_final,overall_risk_score
0,DEPT-1133,Digital Governance,Financial_Reporting,Corporate_HQ,Compliance,8.0,Urban,Large,Centralized,2.0,...,4.0,NaN,2.0,2.0,1.0,26.0,NaN,40.0,NaN,43.0
1,DEPT-4312,West Internal Controls,Financial_Reporting,Corporate_HQ,Operations,6.0,Urban,Small,Centralized,1.0,...,4.0,2.0,1.0,1.0,5.0,2.0,45.0,45.0,45.0,11.0
2,DEPT-3347,Business Integrity - Central Region,Financial_Reporting,Corporate_HQ,Operations,6.0,Urban,Small,Centralized,2.0,...,4.0,4.0,3.0,5.0,5.0,6.0,45.0,50.0,50.0,53.0
3,DEPT-2554,Central Team,Financial_Reporting,Corporate_HQ,Support,5.0,Urban,Small,Autonomous,3.0,...,3.0,4.0,4.0,4.0,1.0,0.0,65.0,60.0,65.0,40.0
4,DEPT-3151,Compliance - East Region,Financial_Reporting,Corporate_HQ,None,8.0,Urban,Small,Centralized,3.0,...,2.0,4.0,NaN,4.0,4.0,10.0,50.0,50.0,50.0,35.0
